In [1]:
import numpy as np
import rosenpymodel.cvffnn as mynn
import dataset.beamforming as dt
import rosenpymodel.rpoptimizer as opt
from rputils import actFunc, initFunc
import pruningQuant 


In [2]:
def setData():
    """
    Set up the data for training.

    Returns:
        tuple: Tuple containing the normalized input and output datasets.
    """
    f = 850e6
    SINRdB = 20
    SNRdBs = 25
    SNRdBi = 20
    phi = [1, 60, 90, 120, 160, 200, 240, 260, 280, 300, 330]
    theta = [90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90]
    desired = [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
    modulation = ["QAM", "WGN", "QAM", "PSK", "QAM", "WGN", "QAM", "WGN", "QAM", "PSK", "PSK"]
    Mmod = [4, 0, 64, 8, 256, 0, 16, 0, 64, 16, 8]

    lenData = int(1e4)

    # Converting 'desired' to a NumPy array
    desired = np.array(desired)

    
    # Calling the create_dataset_beam function
    SetIn, SetOut = dt.create_dataset_beam(modulation, Mmod, f, phi, theta, desired, lenData, SINRdB, SNRdBs, SNRdBi)
    
    return SetIn, SetOut

###############################################################################################################
###############################################################################################################


# Initialize input_data and output_data using the setData function
input_data, output_data = setData()

input_dim, output_dim = input_data.shape[1], output_data.shape[1]

In [ ]:

nn = mynn.CVFFNN()
nn.addLayer(ishape=input_dim, neurons=20, 
            weights_initializer=initFunc.random_normal, 
            bias_initializer=initFunc.random_normal, 
            activation=actFunc.tanh)
nn.addLayer(neurons=10, 
            weights_initializer=initFunc.random_normal, 
            bias_initializer=initFunc.random_normal, 
            activation=actFunc.tanh)
nn.addLayer(neurons=20, 
            weights_initializer=initFunc.random_normal, 
            bias_initializer=initFunc.random_normal, 
            activation=actFunc.tanh)
nn.addLayer(neurons=10, 
            weights_initializer=initFunc.random_normal, 
            bias_initializer=initFunc.random_normal, 
            activation=actFunc.tanh)
nn.addLayer(neurons=output_dim, 
            weights_initializer=initFunc.random_normal, 
            bias_initializer=initFunc.random_normal, 
            activation=actFunc.tanh)


In [4]:
nn.fit(input_data, output_data, epochs=500, verbose=100, batch_size=100, optimizer=opt.CVAdamax())

In [ ]:
y_pred = nn.predict(input_data)
print('Accuracy: training {:.2f}%'.format(nn.accuracy(output_data, y_pred)))

In [6]:
weights = pruningQuant.getWeightsFlat(nn)

In [ ]:

pruningQuant.changeWeightsFlat(nn, weights)
pruningQuant.thresholdMagntudeBasedPruning(nn, 0.3, 0.9)
y_pred = nn.predict(input_data)
print('Accuracy: training {:.2f}%'.format(nn.accuracy(output_data, y_pred)))

In [ ]:
pruningQuant.changeWeightsFlat(nn, weights)
pruningQuant.mixedPruningShell(nn, 0.3, 0.8)
y_pred = nn.predict(input_data)
print('Accuracy: training {:.2f}%'.format(nn.accuracy(output_data, y_pred)))

In [ ]:
pruningQuant.changeWeightsFlat(nn, weights)
pruningQuant.quantizeShell(nn, 8, -1, 1)
y_pred = nn.predict(input_data)
print('Accuracy: training {:.2f}%'.format(nn.accuracy(output_data, y_pred)))